In [ ]:
import json
from flask import Flask
from flask import request
import pandas as pd
import io
from tqdm import tqdm
import yfinance as yf
from pandas_market_calendars import get_calendar
import requests

In [ ]:
app = Flask(__name__)

@app.route('/watchlist',methods=["POST","GET"])
def watchlist():
    
    
    dataframe_1d = pd.DataFrame(columns=['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits', 'ticker'])
    dataframe_1d.index = dataframe_1d['Datetime']
    dataframe_1d = dataframe_1d.drop(['Datetime'], axis=1)
    
    url = 'http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqlisted.txt'
    response = requests.get(url)
    data = response.content.decode('utf-8')

    nasdaq_tickers = pd.read_csv(io.StringIO(data), delimiter='|')
    nasdaq_symbols = nasdaq_tickers['Symbol'].tolist()[:-1]

    stocks_not_availiable = []

    for i in tqdm(nasdaq_symbols[:200]):

        try:
            df = fetch_data(i,interval='1d',period='15d')

            if len(df) > 10:
                dataframe_1d = pd.concat([dataframe_1d,df])

            else:
                print("Skipped: ",i)
                stocks_not_availiable.append(i)

        except:
            stocks_not_availiable.append(i)

    filtered_1d = pd.DataFrame(columns=['ticker','open','close','change'])
    
    tickers = list(pd.unique(dataframe_1d['ticker']))

    required = [i for i in tickers if i not in list(filtered_1d['ticker'])]

    for ticker in tqdm(required):

        open_price = dataframe_1d[dataframe_1d['ticker']==ticker]['Open'].values[0]
        close_price = dataframe_1d[dataframe_1d['ticker']==ticker]['Close'].values[-1]

        change = ((close_price-open_price)/open_price)*100

        row = {
            'ticker':ticker,
            'open':open_price,
            'close':close_price,
            'change':change
        }

        filtered_1d = filtered_1d.append(row, ignore_index=True)
        
    
    filtered_1d = filtered_1d.sort_values(['change'], ascending=True)
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    

@app.route('/')
def index():
    return "Watchlist Generator"



def chandelier_exit(timestamp, close, high, low, length=1, mult=2, use_close=True):

    atr = pd.Series(np.nan, index=high.index)  # initialize True Range series with NaNs
    
    for i in range(len(high.values)):
        atr[i] = 2 * max(high.values[i] - low.values[i], abs(high.values[i] - close.values[i-1]), abs(low.values[i] - close.values[i-1]))

    atr = atr.dropna()

    long_stop_list = list()
    prev_long_stop_list = list()

    short_stop_list = list()
    prev_short_stop_list = list()

    dirs = list()

    bought_first = False
    
    result = {}
        
    for i in range(len(atr)):
        ls = close.values[i] - atr[i]

        if i == 0:
            prev_ls = ls
            prev_long_stop_list.append(ls)
            dirs.append(1)
        else:
            prev_ls = long_stop_list[i-1]
            prev_long_stop_list.append(prev_ls)

            if close.values[i-1] > prev_ls:
                ls = max(ls, prev_ls)
            else:
                ls = ls

        long_stop_list.append(ls)

        # Short Stop
        ss = close.values[i] + atr[i]

        if i == 0:
            prev_ss = ss
            prev_short_stop_list.append(ss)
        else:
            prev_ss = short_stop_list[i-1]
            prev_short_stop_list.append(prev_ss)

            if close.values[i-1] < prev_ss:
                ss = min(ss, prev_ss)
            else:
                ss = ss

        short_stop_list.append(ss)

        if close.values[i] > prev_ss:
            dirs.append(1)
        elif close.values[i] < prev_ls:
            dirs.append(-1)
        elif i != 0:
            dirs.append(dirs[-1])

        buy_signal = False
        sell_signal = False

        if dirs[-1] == 1 and dirs[i-1] == -1:
            buy_signal = True
        elif dirs[-1] == -1 and dirs[i-1] == 1:
            sell_signal = True
                    
        if buy_signal:
            bought_first = True
            result[timestamp.values[i]] = {"Buy":True, "Sell":False}
        
        elif sell_signal:
            bought_first = False
            result[timestamp.values[i]] = {"Buy":False, "Sell":True}
            
        else:
            result[timestamp.values[i]] = {"Buy":False, "Sell":False}
            
    return result


def fetch_data(t,interval="1h", period='15d'):
    ticker = yf.Ticker(t)
    hourly_data = ticker.history(interval=interval, period=period)
    hourly_data['ticker'] = [t]*len(hourly_data["Open"])
    return hourly_data

app.run()